In [2]:
import re
from konlpy.tag import Hannanum
from pprint import pprint
import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

In [3]:
data_glam = pd.read_excel("./data/글램.xlsx")
data_dang = pd.read_excel("./data/당연시.xlsx")
data_sim = pd.read_excel("./data/심쿵.xlsx")
data_amanda = pd.read_excel("./data/아만다.xlsx")
data_wipi = pd.read_excel("./data/위피.xlsx")
data_noon = pd.read_excel("./data/정오의데이트.xlsx")

In [4]:
data_glam.head()

,DATE,STAR,REVIEW
0,2020년 11월 2일,별표 5개 만점에 4개를 받았습니다.,글램 2년전에 깔았을때는 젬을 무료로 꾸준히 얻을수가 있었다. 그래서 부족하지만 긴...
1,2020년 11월 1일,별표 5개 만점에 1개를 받았습니다.,"와 나 정말 이런거 안남기는데 돈써서 결제해서 매칭되면 하나같이 레파토리가똑같다 ""..."
2,2020년 11월 0일,별표 5개 만점에 5개를 받았습니다.,유투브에서 하도 광고틀길래 깔아서 써봤습니다. 솔직하게 다른 곳 보다 과금이 적긴합...
3,2020년 11월 1일,별표 5개 만점에 5개를 받았습니다.,"추천 알고리즘이 훌륭합니다. 다만, 제 경험으로는 연결후 대화가 없는 경우가 열에 ..."
4,2020년 11월 2일,별표 5개 만점에 1개를 받았습니다.,1년전 무료로 이용했을땐 너무 잘 이용 하고있었습니다 그래서 그전에 도움 받은것도있...


In [10]:
print("글램:",len(data_glam))
print("당연시:",len(data_dang))
print("심쿵:",len(data_sim))
print("아만다:",len(data_amanda))
print("위피:",len(data_wipi))
print("정오의 데이트:",len(data_noon))

글램: 24431
당연시: 19973
심쿵: 30312
아만다: 17754
위피: 13183
정오의 데이트: 34287


In [11]:
## 글램+위피 up/ 나머지down

In [21]:
data_up = pd.concat([data_sim, data_wipi])
len(data_up)

43495

In [22]:
data_down = pd.concat([data_glam, data_dang,data_amanda,data_noon])
len(data_down)

96445

In [63]:
data_up.to_excel('글램+위피.xlsx')
data_down.to_excel('심쿵+당연시+아만다+정오.xlsx')

### 다시

In [100]:
data_up = pd.read_excel("./data/글램+위피.xlsx")
data_down = pd.read_excel("./data/심쿵+당연시+아만다+정오.xlsx")

In [101]:
print(len(data_up))
print(len(data_down))

39349
85079


In [102]:
del data_up['Unnamed: 0']
del data_down['Unnamed: 0']

In [103]:
data_up.head()

,DATE,STAR,REVIEW
0,2020년 10월 30일,1,"좋아요 따로, 하트 따로... 솔직히 말씀드려서 돈벌려고 만든 앱 그 이상도 이하도..."
1,2020년 10월 14일,1,이앱 절대 하지마세요 모든것이 하트구매해야 프로필도 볼수있고 다 돈과연결되어있습니다...
2,2020년 10월 18일,1,ㅋㅋㅋㅋㅋㅋㅋ 현질했는데도 매칭 안됌 내가 못생긴 것도 아닌데 이것도 중국식 가짜앱...
3,2020년 10월 14일,1,제 주관적인 평가입니다 우선 매칭이 심플한 점이 장점이라고 생각합니다. 하지만 매칭...
4,2020년 7월 5일,3,다른앱을 쓰다 이 앱을 알게되어 일주일째 쓰고 있는데 아쉬운점이 있어 글남깁니다. ...


In [104]:
data_up['DATE']=data_up['DATE'].str.replace('년','')
data_up['DATE']=data_up['DATE'].str.replace('월','')
data_up['DATE']=data_up['DATE'].str.replace('일','')
data_up['DATE']=data_up['DATE'].str.replace(" ",'')
data_down['DATE']=data_down['DATE'].str.replace('년','')
data_down['DATE']=data_down['DATE'].str.replace('월','')
data_down['DATE']=data_down['DATE'].str.replace('일','')
data_down['DATE']=data_down['DATE'].str.replace(" ",'')

In [105]:
data_up.head(2)

,DATE,STAR,REVIEW
0,20201030,1,"좋아요 따로, 하트 따로... 솔직히 말씀드려서 돈벌려고 만든 앱 그 이상도 이하도..."
1,20201014,1,이앱 절대 하지마세요 모든것이 하트구매해야 프로필도 볼수있고 다 돈과연결되어있습니다...


In [106]:
data_up['DATE']=pd.to_numeric(data_up['DATE'])
data_down['DATE']=pd.to_numeric(data_down['DATE'])

In [107]:
data_up_1 = data_up['DATE'] <= 20181231
data_up_2 = data_up['DATE'] >= 20190101

In [108]:
data_up_1 = data_up[data_up_1]
data_up_2 = data_up[data_up_2]

In [109]:
data_up_1

,DATE,STAR,REVIEW
4,202075,3,다른앱을 쓰다 이 앱을 알게되어 일주일째 쓰고 있는데 아쉬운점이 있어 글남깁니다. ...
5,2020831,1,오늘 개빡쳐서 별점 1개로 수정하고 장문의 사유서 쓰겠습니다. 1. 남자 중년 아재...
6,202097,1,이 어플 ㅈ되는 어플이네요 결제했더니 하트 안들어와서 문의했드만 결제확인이 안된다는...
7,2020925,1,밑에 알바들에게 속지 마세요 진짜 X같네요 제가 왠만해서 이런거 절대 안올리는데 이...
8,2020630,5,가슴이 웅장해집니다. 남녀 다 취급하는 포주 수준. 유저 경험과 회사 이익의 절충선...
...,...,...,...
39344,2019311,4,좋아요
39345,2019114,5,좋아요
39346,201929,4,좋네요
39347,2019212,4,좋아요


In [110]:
data_down_1 = data_down['DATE'] <= 20181231
data_down_2 = data_down['DATE'] >= 20190101
data_down_1 = data_down[data_down_1]
data_down_2 = data_down[data_down_2]

In [111]:
data_down_2

,DATE,STAR,REVIEW
1,20201021,3,다른앱보다 괜찮은데 지금 문제가 심각한게 허위프로필로 사기치려는 계정이 심각하게 늘...
2,20201020,3,무료로 사용할 수 있다는 점이 굉장히 좋구요. 최근 업데이트를 해서인지 높은 평가해...
4,20201031,1,안녕하세요 몇몇 어플써본사람 입니다 확실히 타 어플들에 비해 결제요구 가 덜한느낌도...
6,20201015,5,다른 소개팅 어플 다 써봤어요. 다 지우고 이거 하나 남았습니다. 연결이 자주 되진...
7,20201016,5,글램은 제가 사용해본 어플중에 최고의 어플입니다. 수많은 다른어플들은 매칭도 서울 ...
...,...,...,...
82813,20191010,5,긍정적
82815,20191011,5,좋아요
82819,20191014,4,아아아
82821,20191013,5,좋아여


In [112]:
print(len(data_up_1))
print(len(data_up_2))
print(len(data_down_1))
print(len(data_down_2))

36286
3063
81544
3535


In [113]:
data_up_1.to_excel('글램+위피 전.xlsx')
data_up_2.to_excel('글램+위피 후.xlsx')
data_down_1.to_excel('심쿵+당연시+아만다+정오 전.xlsx')
data_down_2.to_excel('심쿵+당연시+아만다+정오 후.xlsx')

# 여기부터 다시 돌리기

In [2]:
import re
from konlpy.tag import Hannanum
from pprint import pprint
import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

In [3]:
doc = pd.read_excel("./data/글램+위피 전.xlsx")

In [4]:
doc=doc.reset_index(drop=True)

In [5]:
doc=doc.dropna(axis=0) # 결측치있는 행 제거
    
data_date=doc['DATE'] # 리뷰 날짜
data_content=doc['REVIEW'][0:2] # 리뷰 콘텐츠

In [7]:
def review_preprocessing(data):
    # Hannanum package
    pos_tagger = Hannanum()

    # 뉴스를 tokenizing한 후, 명사만 추출
    pos_nouns = pos_tagger.nouns(data)
    
    return ' '.join(pos_nouns)

In [8]:
content_noun=[]
for i in data_content:
    content_noun.append(review_preprocessing(i))

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

# ============================================
# -- Get TFIDF
# ============================================
vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(content_noun)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx
documents=[]
sentences=[]
values=[]
for i, sent in enumerate(content_noun):
    for token in sent.split():
        documents.append(i)
        sentences.append(token)
        values.append(sp_matrix[i, word2id[token]])
    
#     print('====== document[%d] ======' % i)
#     print( [ (token, sp_matrix[i, word2id[token]]) for token in sent.split() ] )

In [29]:
res_dict = []
for i in range(len(values)):
    res_dict.append({
        'documents' : documents[i],
        'sentences' : sentences[i],
        'values' : values[i]
    })
    
res_df = pd.DataFrame(res_dict)
res_df

,documents,sentences,values
0,0,다른앱,0.156102
1,0,앱,0.000000
2,0,일주일,0.156102
3,0,아쉬운점,0.156102
4,0,글남깁니,0.156102
...,...,...,...
93,1,초이스,0.147382
94,1,뭐,0.147382
95,1,메시,0.147382
96,1,전체,0.104863
